In [118]:
import os
import json
import ollama
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

load_dotenv()

pc = Pinecone(api_key="pcsk_4cH8kP_SEu6dcG2q2iPPFW8HwG1famWWy5BYNccqVdK7kXLezc6YAFfW8GTdsLZS96nRH8")

# Ollama model configuration
OLLAMA_EMBEDDING_MODEL = "nomic-embed-text"  # Or another compatible embedding model

# Ollama embeddings function
def generate_ollama_embedding(text, model=OLLAMA_EMBEDDING_MODEL):
    try:
        response = ollama.embeddings(
            model=model,
            prompt=text
        )
        return response['embedding']
    except Exception as e:
        print(f"Error generating Ollama embedding: {e}")
        return None

INDEX_NAME = "catalog"

def create_or_get_index(index_name=INDEX_NAME, dimension=768): # nomic-embed-text is 768-d
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"Created new Pinecone index: {index_name}")
    return pc.Index(index_name), index_name

index, index_name = create_or_get_index()

def upsert_products(index, index_name, products, namespace):
    vectors = []
    for product in products:
        product_text = f"{product['name']} {product.get('color', 'Not specified')} {' '.join(product.get('keyFeatures', []))}"
        embedding = generate_ollama_embedding(product_text)
        if embedding is None:
            print(f"Failed to generate embedding for product: {product['name']}")
            continue
        simplified_metadata = {
            "text": product_text,
            "name": product['name'],
            "price": str(product.get('price', 'Not specified')),
            "color": product.get('color', 'Not specified') if product.get('color') is not None else 'Color not specified',
            "keyFeatures": ', '.join(product.get('keyFeatures', []))
        }
        vectors.append({
            "id": product['productId'],
            "values": embedding,
            "metadata": {k: v for k, v in simplified_metadata.items() if v is not None}
        })
    if vectors:
        index.upsert(vectors=vectors, namespace=namespace)
        print(f"Upserted {len(vectors)} products into index {index_name} under namespace {namespace}")
    else:
        print("No vectors to upsert.")

with open('catalog.json', 'r') as f:
    data = json.load(f)
    stores_data = data['stores']

print(f"Total stores found: {len(stores_data)}")

for store_info in stores_data:
    store_name = store_info['store']
    products = store_info['products']
    namespace = store_name.lower().replace(' ', '_')
    print(f"Processing store: {store_name} with {len(products)} products")
    upsert_products(index, index_name, products, namespace)


Total stores found: 5
Processing store: claires with 10 products
Upserted 10 products into index catalog under namespace claires
Processing store: pacsun with 10 products
Upserted 10 products into index catalog under namespace pacsun
Processing store: americaneagle with 10 products
Upserted 10 products into index catalog under namespace americaneagle
Processing store: icing with 10 products
Upserted 10 products into index catalog under namespace icing
Processing store: walmart with 10 products
Upserted 10 products into index catalog under namespace walmart


In [ ]:
INDEX_NAME = "stores-list"

def create_or_get_index(index_name=INDEX_NAME, dimension=1024):
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"Created new Pinecone index: {index_name}")
    return pc.Index(index_name), index_name

index, index_name = create_or_get_index()

def upsert_store_descriptions(index, index_name, stores_data, batch_size=100):
    total_stores = len(stores_data)
    num_batches = math.ceil(total_stores / batch_size)

    for batch_num in range(num_batches):
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, total_stores)
        batch_stores = stores_data[start_idx:end_idx]

        vectors = []
        for store_info in batch_stores:
            store_name = store_info['store']
            store_description = store_info['description']
            embedding = bedrock_embeddings.embed_query(store_description)
            metadata = {
                "store_name": store_name,
                "description": store_description  
            }
            vectors.append({
                "id": f"{store_name}",
                "values": embedding,
                "metadata": metadata
            })

        index.upsert(vectors=vectors)
        print(f"Upserted batch {batch_num + 1}/{num_batches} with {len(vectors)} store descriptions into index {index_name}")

    print(f"Completed upserting all {total_stores} store descriptions")

with open('stores_list.json', 'r') as f:
    data = json.load(f)
    stores_data = data['stores']

print(f"Total stores found: {len(stores_data)}")
upsert_store_descriptions(index, index_name, stores_data, batch_size=2)


: 

In [122]:
INDEX_NAME = "stores-list"
index_s = pc.Index(INDEX_NAME)

: 

In [94]:
def get_store_recommendations(profile_description, index=index_s, top_k=5):
    embedding_body = json.dumps({
        "inputText": profile_description
    })
    
    embedding_response = bedrock_runtime.invoke_model(
        body=embedding_body,
        modelId=embedding_model_id,
        accept="application/json",
        contentType="application/json"
    )
    
    embedding_result = json.loads(embedding_response['body'].read())
    query_embedding = embedding_result["embedding"]  
    
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    if results['matches']:
        store_descriptions = [
            f"{match['metadata']['store_name']} - {match['metadata'].get('description', 'No description available.')[:200]}..."
            for match in results['matches']
        ]
        
        prompt_body = json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 450,
            "messages": [
                {
                    "role": "user",
                    "content": f"Considering the profile: {profile_description}. Here are some stores: {', '.join(store_descriptions)}. "
                               "Based on this profile, please suggest the three most suitable stores, including reasons for each choice. "
                               "Please stick to the given stores, don't generate new ones."
                }
            ]
        })
        
        response = bedrock_runtime.invoke_model(
            body=prompt_body,
            modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            accept="application/json",
            contentType="application/json"
        )
        
        response_body = json.loads(response['body'].read())

        if 'content' in response_body:
            formatted_output = ""
            for content_block in response_body["content"]:
                if content_block["type"] == "text":
                    formatted_output += content_block["text"] + "\n\n"
            
            return formatted_output.strip()  
        else:
            print(json.dumps(response_body, indent=2))  
            return "Unexpected response format from Claude model."
    else:
        print("No suitable stores were found based on your preferences.")
        return "No suitable stores were found based on your preferences."
        
       


In [101]:
import re

def extract_store_names(recommendation_text):
    store_names = []
    
    matches = re.findall(r"##\s*(\d+)\.\s*(.+)", recommendation_text)
    
    for match in matches:
        store_name = match[1].strip().lower().replace("'", "").replace(" ", "")  
        store_names.append(store_name)
    
    return store_names

In [ ]:
profile_description = "A teenage girl who loves trendy fashion accessories, enjoys experimenting with makeup, and is looking for affordable clothing options."
recommendation_text = get_store_recommendations(profile_description)
print("Store Recommendations:\n")
print(recommendation_text)

Store Recommendations:

# Top 3 Store Recommendations for a Fashion-Forward Teen

Based on the profile of a teenage girl who loves trendy fashion accessories, makeup experimentation, and affordable clothing, here are the three most suitable stores from the given options:

## 1. Claire's
Claire's is perfect for this teen because it specifically targets young women and teenagers with trendy and affordable accessories. Their extensive range of jewelry, hair accessories, and beauty products will appeal to her love of fashion accessories. As someone experimenting with makeup, she'll appreciate Claire's selection of cosmetics that are designed for younger users and come at budget-friendly prices.

## 2. American Eagle
American Eagle offers casual, trendy clothing that resonates with teen fashion sensibilities. While known for their quality denim, they also provide a variety of affordable clothing options that would appeal to a budget-conscious teenager. Their style is age-appropriate yet fas

In [124]:
new_index="catalog"
index_p = pc.Index(new_index)

def get_recommendations(profile_description, index=index_p, top_k=5):
    INDEX_NAME = "stores-list"
    index_s = pc.Index(INDEX_NAME)


    recs = get_store_recommendations(profile_description, index=index_s)
    if recs is None:
        return "Unable to determine recs from the profile description."
    
    print("Store Recommendations:\n")
    print(recs)
    
    namespaces = extract_store_names(recs)
    
    embedding_body = json.dumps({
        "inputText": profile_description
    })
    
    embedding_response = bedrock_runtime.invoke_model(
        body=embedding_body,
        modelId=embedding_model_id,
        accept="application/json",
        contentType="application/json"
    )
    
    embedding_result = json.loads(embedding_response['body'].read())
    query_embedding = embedding_result["embedding"]  
    
    for namespace in namespaces:
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True,
            namespace=namespace
        )
        
        if results['matches']:
            product_descriptions = [
                f"{match['metadata']['name']} - {match['metadata'].get('description', 'No description available.')}"
                for match in results['matches']
            ]
            
            prompt_body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 450,
                "messages": [
                    {
                        "role": "user",
                        "content": f"Considering the profile: {profile_description}. Here are some products from {namespace}: {', '.join(product_descriptions)}. "
                                   "Based on this profile, please suggest only three products that would be most suitable, including reasons for each choice. "
                                   "Please stick to the given products, don't generate new ones."
                    }
                ]
            })
            
            response = bedrock_runtime.invoke_model(
                body=prompt_body,
                modelId="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
                accept="application/json",
                contentType="application/json"
            )
            
            response_body = json.loads(response['body'].read())

            if 'content' in response_body:
                formatted_output = ""
                for content_block in response_body["content"]:
                    if content_block["type"] == "text":
                        formatted_output += content_block["text"] + "\n\n"
                
                lines = formatted_output.split('\n')
                structured_response = ""
                count = 1
                for line in lines:
                    clean_line = line.strip()
                    clean_line = re.sub(r'^\d+\.\s*', '', clean_line)  
                    clean_line = re.sub(r'^##\s*\d+\.\s*', '', clean_line)  
                    if clean_line:
                        if count == 1:
                            structured_response += f"{clean_line}\n"
                        else:
                            structured_response += f"{clean_line}\n"
                        count += 1

                structured_response = '\n'.join([line for line in structured_response.split('\n') if line.strip()])

                print("\n")  
                print(f"Recommendations for {namespace}:\n")
                print(structured_response.strip())
            else:
                print(json.dumps(response_body, indent=2)) 
                print(f"Unexpected response format from Claude model for {namespace}.")
        else:
            print(f"No suitable products were found in the {namespace} store based on your preferences.")

profile_description = "A girl of age 3, who likes the color yellow, is super energetic and likes to talk to people."
get_recommendations(profile_description)


Store Recommendations:

# Best Store Recommendations for a 3-Year-Old Girl

Based on the profile of an energetic 3-year-old girl who likes yellow and is social, here are the three most suitable stores from the options provided:

## 1. Walmart
Walmart is ideal because it offers:
- A wide selection of children's clothing, toys, and accessories in various colors including yellow
- Age-appropriate options specifically designed for toddlers and preschoolers
- Affordable prices for quickly outgrown children's items
- A diverse shopping environment that can engage an energetic child

## 2. Claire's
Claire's works well because:
- They carry colorful accessories and jewelry that appeal to young children
- Their children's section includes yellow items and bright, playful accessories
- The store offers small, fun items perfect for rewards for an energetic 3-year-old
- The interactive nature of the store allows a social child to engage with colorful displays

## 3. Walmart (again, as other option

In [126]:
profile_description = "A woman who is a mother of two kids, one is 3 years old and the other is 6, wants to shop for them."
get_recommendations(profile_description)

Store Recommendations:

# Recommended Stores for a Mother of Two Young Children (Ages 3 and 6)

## 1. Walmart
Walmart would be an excellent choice for this mother because it offers comprehensive shopping solutions for children of all ages. The store provides:
- Affordable children's clothing, shoes, and accessories for both a 3-year-old and a 6-year-old
- Toys appropriate for different developmental stages
- Essential items like children's toiletries, snacks, and school supplies for the 6-year-old
- The convenience of one-stop shopping, which is valuable for a busy mother of young children

## 2. Claire's
Claire's would be particularly suitable because:
- It offers age-appropriate accessories for children, especially appealing to the 6-year-old child who may be developing fashion preferences
- They carry fun hair accessories, jewelry, and small toys that are designed specifically for children
- The store has affordable options for children's birthday party gifts and special occasions
-

## Testing the llm

In [127]:
profile_description = "A woman who is a mother of two kids, one is 3 years old and the other is 6, wants to shop for them. She has an income of $2000/month."
get_recommendations(profile_description)

Store Recommendations:

# Store Recommendations for a Mother of Two Young Children

Based on the profile of a mother with two children (ages 3 and 6) and a monthly income of $2000, here are the three most suitable stores from the given options:

## 1. Walmart
- **Budget-friendly**: With a limited monthly income of $2000 and two growing children, Walmart's commitment to affordability is essential for stretching her budget.
- **One-stop shopping**: She can purchase clothing, toys, groceries, and household essentials in a single trip, saving valuable time for a busy mom.
- **Wide selection**: Walmart offers age-appropriate items for both a 3-year-old and 6-year-old, from clothing to toys to school supplies.

## 2. Claire's
- **Affordable accessories**: Claire's offers trendy yet inexpensive accessories that would appeal to her 6-year-old child, especially if it's a girl.
- **Special occasion items**: Perfect for finding small rewards, birthday gifts, or special treats that won't break her

In [125]:
profile_description = "A teenage girl who loves trendy fashion accessories, enjoys experimenting with makeup, and is looking for affordable clothing options."
get_recommendations(profile_description)

Store Recommendations:

# Top Three Store Recommendations for a Trendy Teen on a Budget

## 1. Claire's
Claire's is ideal for this teen because it specifically targets young women and teenagers with trendy fashion accessories at affordable price points. Their constantly rotating selection of jewelry, hair accessories, and beauty products aligns perfectly with her interest in experimenting with makeup and staying on trend without breaking the bank.

## 2. Pacsun
Pacsun would appeal to this fashion-conscious teen as it offers trendy, California-inspired clothing that's popular among teenagers. While slightly higher-priced than some options, Pacsun frequently has sales and their stylish clothing options would satisfy her desire for trendy pieces. Their selection includes affordable accessories that complement their clothing lines.

## 3. Walmart
For a teen specifically looking for affordability, Walmart is an excellent choice. They offer budget-friendly clothing options and have expanded 

In [128]:
profile_description = "A single parent with a toddler who loves colorful toys and educational books. They are looking for safe and affordable products for their child."
get_recommendations(profile_description)

Store Recommendations:

# Store Recommendations for Single Parent with a Toddler

Based on your profile of a single parent with a toddler looking for safe, affordable, and educational/colorful toys, I recommend these three stores from your list:

## 1. Walmart
Walmart is the most suitable option because it offers:
- Wide range of affordable children's products that fit a single parent's budget
- Dedicated toy section with colorful, educational options for toddlers
- Children's books section with age-appropriate educational materials
- Essential baby and toddler items like safety equipment, clothing, and furniture
- One-stop shopping convenience, which is valuable for a busy single parent

## 2. Claire's
Claire's could be a secondary option because:
- They carry colorful accessories that would appeal to a toddler who loves bright items
- They offer affordable children's jewelry and hair accessories
- Some of their products feature child-friendly characters and themes
- They sometimes ca

In [129]:
profile_description = "A grandmother who wants to buy gifts for her two grandchildren, aged 4 and 7, focusing on creative toys and books that encourage imagination."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for Grandmother Shopping for Grandchildren

Based on the grandmother's profile seeking creative toys and books that encourage imagination for her grandchildren (ages 4 and 7), here are the three most suitable stores from the options provided:

## 1. Walmart
Walmart is the clear top choice because it offers a comprehensive selection of children's toys, educational games, craft supplies, and age-appropriate books. The store's extensive product range includes creative building sets, art supplies, science kits, and imaginative play toys that align perfectly with the grandmother's desire to foster creativity and imagination. Walmart's affordability also allows her to purchase multiple quality items within her budget.

## 2. Claire's
While primarily known for accessories, Claire's carries child-friendly creative items like craft kits, activity sets, and themed toys that could appeal particularly to the 7-year-old. They offer products that 

In [130]:
profile_description = "A teenage boy passionate about gaming, looking for gaming accessories and casual streetwear to match his style."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for a Teenage Gamer

Based on the profile of a teenage boy who loves gaming and wants gaming accessories and casual streetwear, here are the three most suitable stores from the options provided:

## 1. PacSun
PacSun's California-inspired aesthetic aligns perfectly with contemporary gaming culture. Their streetwear focus offers exactly the casual style this teen is looking for, featuring brands that often collaborate with gaming franchises and esports teams. The surf-and-skate inspired apparel fits well with gaming lifestyle aesthetics that many teenage gamers prefer.

## 2. Walmart
For a gaming enthusiast, Walmart is an excellent choice due to its extensive electronics department that carries a wide range of gaming accessories at affordable prices. From controllers and headsets to LED lighting for gaming setups, Walmart offers budget-friendly options. Additionally, their casual clothing section provides basic streetwear pieces that c

In [131]:
profile_description = "A high school student who is into sports and fitness, searching for athletic gear and affordable workout clothes."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for High School Athlete

Based on the profile of a high school student interested in sports and fitness who needs athletic gear and affordable workout clothes, here are the three most suitable stores from the given options:

## 1. Walmart
Walmart would be an excellent primary choice because it offers a wide range of affordable athletic wear and fitness equipment. For a high school student likely on a budget, Walmart's commitment to affordability makes it possible to get quality workout clothes and sports gear without spending too much. They carry recognized athletic brands as well as their own budget-friendly options.

## 2. American Eagle
American Eagle offers comfortable, casual clothing that works well for an active lifestyle. While not primarily a sports retailer, they do offer athleisure options that are stylish enough for everyday wear but functional enough for light workouts or post-practice activities. Their clothing appeals 

In [132]:
profile_description = "A college student living on a tight budget who needs affordable furniture and decor for their dorm room along with basic clothing essentials."
get_recommendations(profile_description)

Store Recommendations:

# Most Suitable Stores for a College Student on a Budget

Based on the profile of a college student living on a tight budget who needs affordable furniture, dorm decor, and basic clothing essentials, here are the three most suitable stores from the provided options:

## 1. Walmart
- **Primary reason**: Offers the most comprehensive selection of budget-friendly furniture, dorm essentials, storage solutions, AND clothing basics all in one place
- Provides "one-stop shopping" convenience for nearly everything a dorm room needs
- Features extremely affordable home goods, bedding, desk supplies, and basic clothing essentials
- Carries small appliances and food items for dorm living at budget-friendly prices

## 2. American Eagle
- **Primary reason**: Provides quality, durable casual clothing that balances affordability with longevity
- Offers frequent sales and student discounts that make their clothing more budget-friendly
- Specializes in denim and everyday wear th

In [133]:
profile_description = "A young professional in their mid-20s who enjoys minimalist fashion and eco-friendly products for their apartment."
get_recommendations(profile_description)

Store Recommendations:

# Store Recommendations for Young Professional

Based on the profile of a young professional in their mid-20s who enjoys minimalist fashion and eco-friendly products for their apartment, here are the three most suitable stores from the provided options:

## 1. American Eagle
American Eagle's casual, all-American style aligns well with minimalist fashion preferences. Their high-quality denim and comfortable everyday wear would appeal to a young professional seeking simple yet durable clothing options. Additionally, American Eagle has made strides in sustainability with eco-friendly initiatives and more sustainable denim production, which matches the profile's interest in eco-friendly products.

## 2. Walmart
For a young professional furnishing their apartment with eco-friendly products, Walmart offers an increasingly diverse range of affordable sustainable home goods. Their expanding selection of environmentally conscious products, combined with their accessibili

In [134]:
profile_description = "A couple in their early 30s planning to renovate their home, looking for stylish furniture and home decor within a reasonable budget."
get_recommendations(profile_description)

Store Recommendations:

# Top Store Recommendations for Home Renovation Project

Based on the profile of a couple in their early 30s looking for stylish home renovation items on a reasonable budget, here are the three most suitable stores from the provided options:

## 1. Walmart
Walmart is an excellent choice for home renovations on a budget. Their extensive product range includes furniture, home decor, and renovation supplies at affordable prices. The store's commitment to affordability aligns perfectly with the couple's desire for reasonable pricing, while still offering stylish and contemporary home items that would appeal to people in their early 30s.

## 2. American Eagle
While primarily known for clothing, American Eagle's casual, all-American aesthetic could translate well to certain home decor items that match the couple's stylish preferences. Their high-quality approach might offer unique accent pieces or textiles that could complement a newly renovated space with a modern ye

In [135]:
profile_description = "A fitness enthusiast in their late 20s searching for high-quality gym equipment, protein supplements, and athletic wear to support their active lifestyle."
get_recommendations(profile_description)

Store Recommendations:

# Top Three Store Recommendations for the Fitness Enthusiast

Based on the profile of a fitness enthusiast in their late 20s looking for gym equipment, protein supplements, and athletic wear, here are the most suitable stores from the given options:

## 1. Walmart
- **Primary benefit**: Offers a comprehensive selection of fitness equipment, protein supplements, and athletic wear at competitive prices
- **Relevance**: Their extensive product range allows the fitness enthusiast to find everything they need in one place
- **Value factor**: Their commitment to affordability helps maintain an active lifestyle without breaking the budget, especially for larger gym equipment purchases

## 2. American Eagle
- **Secondary benefit**: Provides high-quality, comfortable athletic-adjacent casual wear that can complement an active lifestyle
- **Relevance**: Their emphasis on comfortable, durable clothing aligns with the needs of someone with an active lifestyle
- **Style fact

In [136]:
profile_description = "A retired individual who enjoys yoga and meditation, looking for comfortable yoga mats, meditation cushions, and wellness products."
get_recommendations(profile_description)

Store Recommendations:

# Store Recommendations for Retired Yoga and Meditation Enthusiast

Based on your profile as a retired individual interested in yoga, meditation, and wellness products, here are the three most suitable stores from the given options:

## 1. Walmart
- **Primary reason**: Walmart offers an extensive selection of yoga mats, meditation cushions, and wellness products at affordable prices, which is beneficial for someone on a retirement budget.
- **Secondary benefit**: Their wide product range includes various comfort levels of yoga mats, meditation accessories, and wellness items like essential oils and supplements.
- **Accessibility**: As a large retailer, Walmart typically has convenient locations and online shopping options for easier access.

## 2. American Eagle
- **Primary reason**: While known for apparel, American Eagle's focus on comfort and quality aligns with the needs of someone practicing yoga regularly.
- **Secondary benefit**: They offer comfortable, s

In [137]:
profile_description = "An artist seeking affordable art supplies like paints, brushes, canvases, and sketchbooks to fuel their creativity on a limited budget."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for an Artist on a Budget

Based on the profile of an artist seeking affordable art supplies on a limited budget, here are the three most suitable stores from the given options:

## 1. Walmart
Walmart is the strongest match for this profile as it offers a comprehensive selection of art supplies at budget-friendly prices. Their commitment to affordability aligns perfectly with the artist's limited budget, while their extensive product range likely includes essential items like paints, brushes, canvases, and sketchbooks. Walmart's large scale allows them to offer both basic supplies for beginners and more specialized materials for developing artists.

## 2. Claire's
While primarily known for accessories, Claire's occasionally carries basic art supplies and stationery items that could serve an artist on a budget. They might offer affordable sketchbooks, colored pencils, or craft supplies that could supplement the artist's toolkit. Their

In [138]:
profile_description = "A musician looking for beginner-friendly musical instruments like guitars or keyboards along with accessories like sheet music stands or tuners."
get_recommendations(profile_description)

Store Recommendations:

# Top 3 Store Recommendations for a Beginner Musician

## 1. Walmart
Walmart is the best choice for a beginner musician because they carry a wide range of affordable musical instruments, including entry-level guitars and keyboards. Their stores typically have a dedicated music section with essential accessories like sheet music stands and tuners. Walmart's commitment to affordability makes it ideal for someone just starting their musical journey who may not want to invest heavily in premium instruments right away.

## 2. American Eagle
While American Eagle is primarily known for clothing, some of their larger locations carry lifestyle products that could include basic musical accessories. They occasionally stock small items like basic tuners or guitar picks as part of their lifestyle merchandise collection. However, they would not be a primary destination for musical instruments themselves.

## 3. PacSun
PacSun occasionally carries music-related merchandise as p

In [139]:
profile_description = "A tech enthusiast who loves gadgets and is searching for the latest smart home devices like smart speakers or wearable tech within a $500 budget."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for Tech Enthusiast

Based on your profile as a tech enthusiast looking for smart home devices and wearables within a $500 budget, here are the three most suitable stores from your list:

## 1. Walmart
**Primary Recommendation**: Walmart offers the widest selection of tech products among the listed stores, including smart speakers from brands like Google, Amazon, and Apple, as well as various wearable tech options from Fitbit, Samsung, and more. Their competitive pricing aligns well with your $500 budget, allowing you to potentially purchase multiple smart home devices rather than just one premium item.

## 2. American Eagle
**Secondary Option**: While primarily a clothing retailer, American Eagle has expanded into tech accessories and some wearable tech products that complement their lifestyle brand. They occasionally carry smart fitness trackers and wireless earbuds that might interest a tech enthusiast, though their selection is m

In [140]:
profile_description = "A gamer looking for ergonomic chairs, gaming keyboards, high-quality headphones, and other gaming accessories to enhance their setup."
get_recommendations(profile_description)

Store Recommendations:

# Best Stores for a Gamer Looking for Gaming Accessories

Based on the profile of a gamer seeking ergonomic chairs, gaming keyboards, high-quality headphones, and other gaming accessories, here are the three most suitable stores from the given options:

## 1. Walmart
Walmart offers a comprehensive selection of gaming accessories at various price points. They carry ergonomic gaming chairs, mechanical keyboards from brands like Razer and Logitech, gaming headsets, and other peripherals essential for a complete gaming setup. Their extensive product catalog and competitive pricing make them an excellent one-stop shop for gamers looking to enhance their setup without breaking the bank.

## 2. American Eagle
Although American Eagle is primarily known for clothing, this would be a distant second choice only because some locations occasionally carry limited gaming-themed apparel or accessories that might appeal to gamers looking to express their passion through fashion.

In [141]:
profile_description = "An affluent shopper in their 40s looking for premium skincare products, designer handbags, and high-quality clothing from luxury brands."
get_recommendations(profile_description)

Store Recommendations:

# Store Recommendations for Affluent 40s Shopper Seeking Luxury Items

Based on the profile of an affluent shopper in their 40s looking for premium skincare, designer handbags, and luxury clothing, here are the most suitable stores from the given options:

## Top Recommendations

Unfortunately, none of the listed stores align well with this shopper's luxury preferences. The available options (Pacsun, Walmart, Claire's, American Eagle, and Icing) are primarily mid-range or budget retailers that don't carry the premium luxury products this shopper seeks.

If I had to select the relatively better options from this list:

1. **American Eagle** - While primarily known for casual wear, they offer higher quality denim and everyday clothing than some other options. However, they don't carry luxury brands or premium skincare.

2. **Pacsun** - Offers some trendy, higher-priced fashion items, but their California-inspired casual aesthetic doesn't align with luxury designer

In [142]:
profile_description = "A couple planning a vacation abroad who want travel essentials like durable luggage sets, travel-friendly clothing, and portable tech gear like power banks or adapters from trusted brands."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for Traveling Couple

Based on your profile of a couple planning a vacation abroad who need travel essentials, here are the three most suitable stores from your list:

## 1. Walmart
Walmart is an excellent choice because it offers comprehensive travel essentials at affordable prices. Their extensive product range includes durable luggage sets, travel adapters, power banks, and practical travel clothing. As a large retailer, Walmart carries trusted brands for electronics and travel gear while providing good value, allowing you to allocate more of your budget to your actual vacation experiences.

## 2. American Eagle
For travel-friendly clothing, American Eagle is ideal. Their casual, comfortable apparel works well for vacation wear, with durable fabrics that can withstand travel conditions. Their jeans, t-shirts, and light outerwear are versatile for different climates and activities while remaining stylish. American Eagle's focus on 

In [143]:
profile_description = "An environmentally conscious shopper searching for sustainable clothing brands, reusable household items like water bottles or bags, and organic food products within a $1000 budget per month."
get_recommendations(profile_description)

Store Recommendations:

# Top 3 Store Recommendations for Environmentally Conscious Shopper

## 1. Walmart
Walmart is ideal for your sustainable shopping needs because:
- They've expanded their eco-friendly product lines with affordable options for organic food products
- You can find a growing selection of reusable household items like water bottles and shopping bags
- Their scale allows them to maintain lower prices on sustainable products, helping you stay within your $1000 monthly budget
- They offer sustainable clothing options through initiatives like recycled materials collections

## 2. American Eagle
American Eagle aligns well with your environmental values because:
- They've implemented significant sustainability initiatives in their clothing production
- Their denim lines include options made with recycled materials and water-saving manufacturing processes
- They offer higher-quality, longer-lasting clothing items that reduce the need for frequent replacements
- While more e

In [144]:
profile_description = "A pet owner shopping for high-quality pet food, toys, grooming supplies, and accessories for their dog or cat while staying within a $200 monthly budget."
get_recommendations(profile_description)

Store Recommendations:

# Top 3 Store Recommendations for a Pet Owner with $200 Monthly Budget

## 1. Walmart
Walmart is the clear top choice for your pet care needs. With its commitment to affordability and extensive product range, Walmart offers comprehensive pet supplies including premium and budget-friendly pet food brands, durable toys, grooming essentials, and accessories. Their competitive pricing would allow you to maximize your $200 monthly budget while still providing quality care for your dog or cat.

## 2. Claire's
While primarily known for fashion accessories, Claire's does carry a selection of pet accessories like cute collars, leashes, and pet-themed items. Their affordable pricing could supplement your main pet shopping, particularly for decorative accessories or seasonal items for your pet. However, their pet selection would be quite limited compared to dedicated pet retailers.

## 3. Icing
Similar to Claire's, Icing occasionally offers trendy pet accessories like styl

In [145]:
profile_description = "An avid reader searching for affordable bookshelves to organize their growing collection of novels along with cozy reading lamps or beanbags to create the perfect reading nook at home."
get_recommendations(profile_description)

Store Recommendations:

# Best Store Recommendations for Avid Reader

Based on your profile as an avid reader looking for affordable bookshelves and cozy reading nook items, here are the three most suitable stores from your list:

## 1. Walmart
Walmart stands out as your best option because of its:
- **Affordability**: Their commitment to low prices aligns perfectly with your need for budget-friendly bookshelves
- **Diverse selection**: They offer a wide range of furniture including various bookshelf styles, sizes, and finishes
- **Complete solution**: You can find both bookshelves AND reading accessories like lamps, throw pillows, and comfortable seating options in one place

## 2. Claire's
While primarily an accessories store, Claire's could help enhance your reading nook with:
- **Affordable decorative elements**: Small items to personalize your reading space
- **Reading accessories**: Potentially offering book lights, bookmarks, and other reading-related items
- **Trendy decor item

In [148]:
profile_description = " "
get_recommendations(profile_description)

Store Recommendations:

I don't see a profile provided in your message that would allow me to make personalized store recommendations. Without information about your preferences, style, age range, or shopping needs, I can't determine which of the listed stores (Walmart, American Eagle, Claire's, PacSun, and Icing) would be most suitable for you.

If you'd like to share some details about yourself or what you're looking for in a shopping experience, I'd be happy to suggest the three most appropriate stores from this list with specific reasons for each recommendation.
